Titanic. 40+ models

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, \
                    cross_val_score

import warnings 
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)                 
warnings.filterwarnings("ignore")

Data pre-processing

In [3]:
# Loading csv
import os
filepath = os.path.abspath(".")
initial_df = pd.read_csv(rf'{filepath}\train.csv')
submission_df = pd.read_csv(rf'{filepath}\test.csv')

# Data preproccesing - Fill NaN values
initial_df['Age'] = initial_df['Age'].fillna(0)
# initial_df['Cabin'] = initial_df['Cabin'].fillna(0)
submission_df['Age'] = submission_df['Age'].fillna(0)
# submission_df['Cabin'] = submission_df['Cabin'].fillna(0)
submission_df['Fare'] = submission_df['Fare'].fillna(0)

# features = ['Pclass', 'Sex', 'Is_Name_Master', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] # + shuffle=False - 0.78708 
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

initial_with_dummies_df = pd.get_dummies(initial_df[features + ['Survived']])
submission_with_dummies_df = pd.get_dummies(submission_df[features])

# Data preproccesing - Get X, y
X = initial_with_dummies_df.drop(['Survived'], axis=1).values
y = initial_with_dummies_df['Survived'].values
X_submission = submission_with_dummies_df.values

# Data preproccesing - Divining Data for training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False, random_state=1)

Let's create a list withh all models. Some models do not have default options, so we will list the options below

In [4]:
# Setup default parameters
estimator = RandomForestClassifier(random_state=1).fit(X_train, y_train)
estimator_kn = LogisticRegression().fit(X_train, y_train)
estimators = [('rf', estimator),
              ('kn', estimator_kn)]
param_grid = {"max_depth": [3, None], "min_samples_split": [5, 10]}

model_names = [
    'AdaBoostClassifier',
    'BaggingClassifier',
    'BernoulliNB',
    'CalibratedClassifierCV',
    ('CategoricalNB', '(min_categories=75)'),
    ('ClassifierChain', f'(base_estimator={estimator_kn})'),    
    'ComplementNB',
    'DecisionTreeClassifier',
    ('DummyClassifier', '(strategy="most_frequent", random_state=1)'),
    'ExtraTreeClassifier',
    'ExtraTreesClassifier',
    'GaussianNB',
    'GaussianProcessClassifier',
    'GradientBoostingClassifier',
    'HistGradientBoostingClassifier',
    'KNeighborsClassifier',
    'LabelPropagation',
    'LabelSpreading',
    'LinearDiscriminantAnalysis',
    'LinearSVC',
    'LogisticRegression',
    'LogisticRegressionCV',
    'MLPClassifier',
    ('MultiOutputClassifier', f'(estimator={estimator})'),
    'MultinomialNB',
    'NearestCentroid',
    'NuSVC',
    ('OneVsOneClassifier', f'(estimator={estimator})'),
    ('OneVsRestClassifier', f'(estimator={estimator})'),
    ('OutputCodeClassifier', f'(estimator={estimator})'),
    'PassiveAggressiveClassifier',
    ('Perceptron', '(shuffle=False, random_state=1)'),
    'QuadraticDiscriminantAnalysis',
    ('RadiusNeighborsClassifier', '(radius=50)'),
    'RandomForestClassifier',
    'RidgeClassifier',
    'RidgeClassifierCV',
    'SGDClassifier',
    'SVC',
    ('StackingClassifier', f'(estimators={estimators})'),
    ('VotingClassifier', f'(estimators={estimators})'),
]

In [5]:
# Useful functions for processing all models
def model_score(model_name, save_to_csv=False):
    model = None
    name = model_name
    if type(model_name) is tuple:
        model = eval(model_name[0] + model_name[1])
        name = str(model).split('(')[0]
    else:
        try:
            model = eval(model_name + '(random_state=1)')
        except:
            model = eval(model_name + '()')
    
    if name == 'ClassifierChain' or name == 'MultiOutputClassifier':
        return model_scores_special(model, save_to_csv=save_to_csv)
    
    model.fit(X_train, y_train)
    
    y_final = model.predict(X_submission)
    final_df = pd.DataFrame({'PassengerId': submission_df['PassengerId'], 'Survived': y_final})
    if save_to_csv:
        final_df.to_csv(f'{name}.csv', index=False)
    
    cv_test = cross_val_score(model, X_test, y_test, cv=5).mean().round(5)
    model_score = model.score(X_test, y_test).round(5)
    return [name, cv_test, model_score]
    
    
def model_scores_special(model, save_to_csv=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False, random_state=1)
    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)
    model.fit(X_train, y_train)
    
    # Calculate model scores
    y_final = model.predict(X_submission)
    y_final = np.concatenate(y_final).astype(int)
    y_final = y_final.reshape(y_final.shape[0],)
      
    final_df = pd.DataFrame({'PassengerId': submission_df['PassengerId'], 'Survived': y_final})
    cv_test = cross_val_score(model, X_test, y_test, cv=5).mean().round(5)
    model_score = model.score(X_test, y_test).round(5)    
    name = str(model).split('(')[0]

    if save_to_csv:
        final_df.to_csv(f'{name}.csv', index=False)
    
    return [name, cv_test, model_score]

Models training

In [ ]:
models_scores = [model_score(name) for name in model_names]  
   
models_scores_df = pd.DataFrame(np.array(models_scores), columns=['Model', 'Cross_Validation', 'Accuracy'])
models_scores_df.sort_values(['Cross_Validation', 'Accuracy'], ascending=False, inplace=True)
models_scores_df = models_scores_df.set_index('Model')


Real scores in Kaggle

In [6]:
models_scores_df['Real_Score'] = [0.76076, 0.77511, 0.77511, 0.77511, 0.77511, 0.77511, 0.77751, 
                                  0.77751, 0.72966, 0.76315, 0.78229, 0.76555, 0.74401, 0.77033,
                                  0.76794, 0.76794, 0.76794, 0.76794, 0.76794, 0.76076, 0.76555,
                                  0.77033, 0.74641, 0.70574, 0.63636, 0.70095, 0.66985, 0.74641,
                                  0.6244 , 0.63397, 0.65789, 0.66507, 0.6555 , 0.66267, 0.62679,
                                  0.65311, 0.64832, 0.61483, 0.61483, 0.61961, 0.622]

models_scores_df

NameError: name 'models_scores_df' is not defined